# Factorised BFGS

This is a small notebook to test the implementation of the factorised BFGS equations.

## 0. Import packages

In [ ]:
import numpy as np

## 1. Input

First, we provide some input that would usually come from previous BFGS iterations. This includes the vectors $\mathbf{s}$ and $\mathbf{y}$, and also the matrices $\mathbf{A}$ and $\mathbf{A}^{-1}$. In the notation used here, $\mathbf{A}$ is the approximation to the *inverse* Hessian.

In [ ]:
# Dimension.
dim=3

# Vectors s and y.
s=np.random.rand(dim)
y=np.random.rand(dim)

print(s)
print(y)

# Matrix A and its inverse.
A=np.identity(dim)
Ainv=np.linalg.inv(A)

## 2. Standard BFGS update

For reference, we compute the standard BFGS update of $\mathbf{A}$.

In [ ]:
# Standard BFGS update. ================================

# Precompute some quantities.
I=np.identity(dim)
rho=1.0/np.dot(s,y)
sy=rho*np.tensordot(s,y,axes=0)
ss=rho*np.tensordot(s,s,axes=0)

# Perform actual BFGS update.
A_new=np.matmul(np.matmul((I-sy),A),(I-sy.transpose()))+ss

# Print some output for checking.
print(A_new)
print(np.linalg.inv(A_new))

## 3. Factorised BFGS update

For the factorised BFGS update, we proceed in several steps: (1) compute $\gamma^2$ and $\beta=\gamma^2 \mathbf{s}^T\mathbf{A}^{-1}\mathbf{s}$, (2) from $\gamma$ and $\beta$ compute $\theta$, (3) compute the auxiliary vectors $\mathbf{a}$ and $\mathbf{b}$, (4) finally compute the vectors $\mathbf{u}$ and $\mathbf{v}$ and the matrix factor $\mathbf{S}=(\mathbf{I}+\mathbf{u}\mathbf{v}^T)$.

The update of the inverse $\mathbf{A}^{-1}$ is then computed from $\mathbf{S}=(\mathbf{I}+\mathbf{u}\mathbf{v}^T)$ using the Sherman-Morrison formula.

In [ ]:
# Partitioned BFGS update of A. ========================

# Precompute some quantities.
gamma2=rho**2 * np.dot(y,np.dot(A,y)) + rho
beta=gamma2 * np.dot(s,np.dot(Ainv,s))
theta=np.sqrt(rho/(beta*gamma2))

# Compute vectors a and b.
a=np.sqrt(gamma2)*s
b=(rho/np.sqrt(gamma2))*np.dot(A,y)

# Compute vectors u and v.
u=a
v=-np.dot(Ainv,b+theta*a)

# Compute partitioned matrix S.
S=I+np.tensordot(u,v,axes=0)

# Compute updated Matrix A_new_f.
A_new_f=np.matmul(S,np.matmul(A,S.transpose()))

# Partitioned BFGS update of Ainv. =====================

# Apply Sherman-Morrison formula to find inverse of S.
Sinv=I-np.tensordot(u,v,axes=0)/(1.0+np.dot(u,v))

# Compute inverse of updated matrix.
Ainv_new_f=np.matmul(Sinv.transpose(),np.matmul(Ainv,Sinv))

# Update determinant. ==================================
det_new=np.linalg.det(A)*(1.0+np.dot(u,v))**2

# Print some output for checking. ======================
print(A_new_f-A_new)
print(Ainv_new_f-np.linalg.inv(A_new))

print(np.linalg.det(A_new_f))
print(det_new)